In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import SimpleITK as sitk
import math
from skimage import morphology
from skimage import measure
from sklearn.cluster import KMeans
from skimage.transform import resize
from tqdm import tqdm
import skimage
from skimage.morphology import ball, disk, dilation, binary_erosion, remove_small_objects, erosion, closing, reconstruction, binary_closing
from skimage.measure import label,regionprops, perimeter
from skimage.morphology import binary_dilation, binary_opening
from skimage.filters import roberts, sobel
from skimage import measure, feature
from skimage.segmentation import clear_border
from skimage import data
from scipy import ndimage as ndi
from glob import glob

In [2]:
def get_filename(case):
  global file_list
  for f in file_list:
    if case in f:
      return(f)

In [3]:

subset="subset9"
# luna_path="C:\\Users\\P70065719\\Desktop\\Thesis\\dataset\\LUNA\\1\\"
luna_path="C:\\Users\\P70065719\\Desktop\\Thesis\\dataset\\LUNA\\1\\"
output_path="C:\\Users\\P70065719\\Desktop\\Thesis\\seq\\"+subset+"\\"
file_list=glob(luna_path+"data\\"+subset+"\\"+"*.mhd")
# file_list=glob(luna_path+subset+"\\"+"*.mhd")
annotations_path=luna_path+"annotations.csv"
# print(annotations_path)
# print(output_path)
# print(luna_path+"data\\"+subset)

In [4]:
df_node=pd.read_csv(annotations_path)
df_node["file"]=df_node["seriesuid"].apply(get_filename)
df_node=df_node.dropna()


In [5]:
def normalizePlanes(npzarray):
        maxHU = 400.
        minHU = -1000.
        npzarray = (npzarray - minHU) / (maxHU - minHU)
        npzarray[npzarray>1] = 1.
        npzarray[npzarray<0] = 0.
        return npzarray

In [6]:
img_list={}
import math
prefix=luna_path+subset+"\\"
f=0

In [7]:
for img_file in file_list:
    #img_file='drive/My Drive/Thesis/dataset/subset0/1.3.6.1.4.1.14519.5.2.1.6279.6001.109002525524522225658609808059.mhd'
    mini_df = df_node[df_node["file"]==img_file] 
    #get all nodules associate with file
    if mini_df.shape[0]>0: # some files may not have a nodule--skipping those 
        # load the data once
        print(f)
        f=f+1
        itk_img = sitk.ReadImage(img_file) 
        img_array = sitk.GetArrayFromImage(itk_img) # indexes are z,y,x (notice the ordering)
        num_z, height, width = img_array.shape        #heightXwidth constitute the transverse plane
        origin = np.array(itk_img.GetOrigin())      # x,y,z  Origin in world coordinates (mm)
        spacing = np.array(itk_img.GetSpacing())
        img_array=normalizePlanes(img_array)
        t_num=-1
        #print(spacing)# spacing of voxels in world coor. (mm)
        # go through all nodes (why just the biggest?)
        for node_idx, cur_row in mini_df.iterrows():       
            node_x = cur_row["coordX"]
            node_y = cur_row["coordY"]
            node_z = cur_row["coordZ"]
            diam = cur_row["diameter_mm"]
            #print(diam)
            # just keep 3 slices
            t_num=t_num+1
            positive_slices=math.ceil(diam/spacing[2])
            imgs = np.ndarray([5,height,width],dtype=np.float32)
            masks = np.ndarray([5,height,width],dtype=np.uint8)
            center = np.array([node_x, node_y, node_z])   # nodule center
            v_center = np.rint((center-origin)/spacing)  # nodule center in voxel space (still x,y,z ordering)
            
            
                
                
            for i, i_z in enumerate(np.arange(int(v_center[2])-2,
                             int(v_center[2])+3).clip(0, num_z-1)): # clip prevents going out of bounds in Z

                flag=0
                if i_z in range(int(v_center[2])-int(positive_slices/2),int(v_center[2])+int(positive_slices/2)):
                    flag=1
                fname=img_file[len(prefix):-4]+"_"+str(t_num)+"_"+str(i_z)+"_"+str(flag)
                img_list[fname] = img_array[i_z]
            
#np.save(os.path.join(output_path,"images_%04d_%04d.npy" % (fcount, node_idx)),imgs)
#             np.save(os.path.join(output_path,"masks_%04d_%04d.npy" % (fcount, node_idx)),masks)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55


In [8]:
f=0
#len(img_list)
import os

In [9]:
for i in img_list:
    print(f)
    f=f+1
    file_path=i.split('_')
    fname=output_path+file_path[0]+"\\"+file_path[1]+"\\"
    if not os.path.exists(fname):
        os.makedirs(fname)
#     fname=output_path+i
   
    img_file=fname+file_path[2]+"_"+file_path[3]+".png"
    
    
    img=img_list[i]

    plt.imsave(img_file,img,cmap=plt.cm.gray)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [10]:
len(img_list)

505